In [7]:
import dis
from malthusia.engine.container.instrument import Instrument

In [8]:
s = """
x = "hej"
y = x * 100
s = {1,2,3}
s2 = {5,6}
s3 = s ^ s2
print(s3)
"""
c = compile(s, "source", "exec")

In [9]:
ci = Instrument.instrument(c, replace_builtins=True, instrument=False, instrument_binary_multiply=True)

In [10]:
new_builtins = {"__builtins__": 
                {"__instrumented_index": lambda x, y: 32, 
                 "type": type, "__safe_type__": type, 
                 "print": print, 
                 "__instrument_binary_multiply__": lambda a,b: print(f"instrumenting {a} * {b}"), 
                 "__build_class__": __build_class__, 
                 "__name__": "DANGEROUS_main"
                }
               }

In [11]:
exec(c, new_builtins)

{1, 2, 3, 5, 6}


In [12]:
exec(ci, new_builtins)

instrumenting hej * 100
{1, 2, 3, 5, 6}


In [7]:
dis.dis(ci)

  2           0 LOAD_CONST               0 ('hej')
              2 STORE_NAME               0 (x)

  3           4 LOAD_NAME                0 (x)
              6 LOAD_CONST               1 (100)
              8 DUP_TOP_TWO
             10 LOAD_GLOBAL              8 (__instrument_binary_multiply__)
             12 CALL_FUNCTION            2
             14 POP_TOP
             16 BINARY_MULTIPLY
             18 STORE_NAME               1 (y)

  4          20 LOAD_CONST               2 (1)
             22 LOAD_CONST               3 (2)
             24 LOAD_CONST               4 (3)
             26 BUILD_SET                3
             28 STORE_NAME               2 (s)

  5          30 LOAD_CONST               5 (5)
             32 LOAD_CONST               6 (6)
             34 BUILD_SET                2
             36 STORE_NAME               3 (s2)

  6          38 LOAD_NAME                2 (s)
             40 LOAD_NAME                3 (s2)
             42 BINARY_XOR
             4